# GFF Processing Metrics

Using a processed GFF file (either from NIH or Ensembl) we can apply some metrics to understand the characteristics of our RNA dataset.

# Libraries

In [5]:
# Libraries


# Combinations
from itertools import combinations

# Floor
import math

# NumPy
import numpy as np

# Pandas
import pandas as pd

# Sampling
# import random  
from random import sample

In [8]:
# Set the data folder.
data_folder = '/media/apollo/Samsung_T5/transfer/mayur/annotations/'
# data_folder = '/home/mad1188/rvallsamples/'

# NIH File

First, read in the processed GFF file from NIH.

In [9]:
# NIH file.

# Skip the metadata lines (1-9).
df = pd.read_csv(
    data_folder + 'GCF_000001405.40_GRCh38.p14_genomic.gff.exon.processed',
    sep = '\t'
)

In [10]:
df

,nih_molecule_accession,source,category,start,stop,strand,information,ID,exon_parent,exon_gbkey,exon_gene,exon_transcript_id,predicted
0,NC_000001.11,BestRefSeq,exon,17369.0,17436.0,-,ID=exon-NR_106918.1-1;Parent=rna-NR_106918.1;D...,exon-NR_106918.1-1,rna-NR_106918.1,precursor_RNA,MIR6859-1,NR_106918.1,no
1,NC_000001.11,BestRefSeq,exon,17369.0,17391.0,-,ID=exon-MIR6859-1-1;Parent=rna-MIR6859-1;Dbxre...,exon-MIR6859-1-1,rna-MIR6859-1,ncRNA,MIR6859-1,NaN,not_available
2,NC_000001.11,BestRefSeq,exon,17409.0,17431.0,-,ID=exon-MIR6859-1-2-1;Parent=rna-MIR6859-1-2;D...,exon-MIR6859-1-2-1,rna-MIR6859-1-2,ncRNA,MIR6859-1,NaN,not_available
3,NC_000001.11,Gnomon,exon,29774.0,30667.0,+,ID=exon-XR_007065314.1-1;Parent=rna-XR_0070653...,exon-XR_007065314.1-1,rna-XR_007065314.1,ncRNA,MIR1302-2HG,XR_007065314.1,yes
4,NC_000001.11,Gnomon,exon,30976.0,31093.0,+,ID=exon-XR_007065314.1-2;Parent=rna-XR_0070653...,exon-XR_007065314.1-2,rna-XR_007065314.1,ncRNA,MIR1302-2HG,XR_007065314.1,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698714,NC_012920.1,RefSeq,exon,14149.0,14673.0,-,ID=exon-ND6-1;Parent=rna-ND6;Dbxref=GeneID:454...,exon-ND6-1,rna-ND6,mRNA,ND6,NaN,not_available
1698715,NC_012920.1,RefSeq,exon,14674.0,14742.0,-,ID=exon-TRNE-1;Parent=rna-TRNE;Dbxref=GeneID:4...,exon-TRNE-1,rna-TRNE,tRNA,TRNE,NaN,not_available
1698716,NC_012920.1,RefSeq,exon,14747.0,15887.0,+,ID=exon-CYTB-1;Parent=rna-CYTB;Dbxref=GeneID:4...,exon-CYTB-1,rna-CYTB,mRNA,CYTB,NaN,not_available
1698717,NC_012920.1,RefSeq,exon,15888.0,15953.0,+,ID=exon-TRNT-1;Parent=rna-TRNT;Dbxref=GeneID:4...,exon-TRNT-1,rna-TRNT,tRNA,TRNT,NaN,not_available


# Metrics

For the metrics that follow, we will concern ourselves only with exons for which a transcript ID is given (including predicted products).

In [11]:
# Only want ones with transcripts.
has_transcript = df[df['predicted'] != 'not_available'].copy()

In [12]:
has_transcript

,nih_molecule_accession,source,category,start,stop,strand,information,ID,exon_parent,exon_gbkey,exon_gene,exon_transcript_id,predicted
0,NC_000001.11,BestRefSeq,exon,17369.0,17436.0,-,ID=exon-NR_106918.1-1;Parent=rna-NR_106918.1;D...,exon-NR_106918.1-1,rna-NR_106918.1,precursor_RNA,MIR6859-1,NR_106918.1,no
3,NC_000001.11,Gnomon,exon,29774.0,30667.0,+,ID=exon-XR_007065314.1-1;Parent=rna-XR_0070653...,exon-XR_007065314.1-1,rna-XR_007065314.1,ncRNA,MIR1302-2HG,XR_007065314.1,yes
4,NC_000001.11,Gnomon,exon,30976.0,31093.0,+,ID=exon-XR_007065314.1-2;Parent=rna-XR_0070653...,exon-XR_007065314.1-2,rna-XR_007065314.1,ncRNA,MIR1302-2HG,XR_007065314.1,yes
5,NC_000001.11,Gnomon,exon,34168.0,35418.0,+,ID=exon-XR_007065314.1-3;Parent=rna-XR_0070653...,exon-XR_007065314.1-3,rna-XR_007065314.1,ncRNA,MIR1302-2HG,XR_007065314.1,yes
6,NC_000001.11,BestRefSeq,exon,30366.0,30503.0,+,ID=exon-NR_036051.1-1;Parent=rna-NR_036051.1;D...,exon-NR_036051.1-1,rna-NR_036051.1,precursor_RNA,MIR1302-2,NR_036051.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698677,NC_000024.10,Gnomon,exon,57192325.0,57192537.0,+,ID=exon-XM_017030055.2-1;Parent=rna-XM_0170300...,exon-XM_017030055.2-1,rna-XM_017030055.2,mRNA,IL9R,XM_017030055.2,yes
1698678,NC_000024.10,Gnomon,exon,57194043.0,57194127.0,+,ID=exon-XM_017030055.2-2;Parent=rna-XM_0170300...,exon-XM_017030055.2-2,rna-XM_017030055.2,mRNA,IL9R,XM_017030055.2,yes
1698679,NC_000024.10,Gnomon,exon,57196336.0,57197337.0,+,ID=exon-XM_017030055.2-3;Parent=rna-XM_0170300...,exon-XM_017030055.2-3,rna-XM_017030055.2,mRNA,IL9R,XM_017030055.2,yes
1698680,NC_000024.10,BestRefSeq,exon,57203182.0,57203350.0,-,ID=exon-NR_138048.1-2-1;Parent=rna-NR_138048.1...,exon-NR_138048.1-2-1,rna-NR_138048.1-2,ncRNA,WASIR1,NR_138048.1,no


First, we will see the breakdown of the various types of RNA.  Note that we are not able to discriminate miRNA, tRNA, etc... from each other with this processing.

In [13]:
# Group and count.

# Keep DataFrame, but we can't use
# .to_frame() because of a header naming
# error, see https://github.com/pandas-dev/pandas/issues/6618

# Source: https://stackoverflow.com/a/42324086
category_counts = has_transcript.groupby('exon_gbkey', as_index = False).size()
category_counts = category_counts.rename(
    columns = {
        "size": "n_category"
    }
)

In [14]:
category_counts

,exon_gbkey,n_category
0,mRNA,1504043
1,misc_RNA,114898
2,ncRNA,73782
3,precursor_RNA,1915
4,rRNA,29


What about genes?  Create a small function to work on the groups.

In [15]:
# Helper function
def category_helper(df_temp):
    
    # Simply count the number of genes.
    return(len(set(df_temp.exon_gene)))

In [16]:
# Group and count unique genes per category.
grouped_and_counted = has_transcript.groupby(['exon_gbkey']).apply(lambda x: category_helper(x))

# We want a DataFrame.
grouped_and_counted = pd.DataFrame(
    {
        'exon_gbkey': list(grouped_and_counted.keys()),
        'n_unique_genes': list(grouped_and_counted.values)
    }
)

/tmp/ipykernel_19944/458226104.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_and_counted = has_transcript.groupby(['exon_gbkey']).apply(lambda x: category_helper(x))


In [17]:
grouped_and_counted

,exon_gbkey,n_unique_genes
0,mRNA,19617
1,misc_RNA,3831
2,ncRNA,12403
3,precursor_RNA,1913
4,rRNA,29


These numbers look about right, so let's move on to predicted vs. non-predicted.

In [18]:
# Get the two letter codes and make a nice dataframe.

# Source: https://www.ncbi.nlm.nih.gov/books/NBK21091/table/ch18.T.refseq_accession_numbers_and_mole/?report=objectonly
two_letter_codes_df = pd.DataFrame(
    data = {
        'category': [i.split('_')[0] for i in list(has_transcript.exon_transcript_id)]
    }
)

two_letter_codes_df = two_letter_codes_df.groupby('category', as_index = False).size()
two_letter_codes_df = two_letter_codes_df.rename(
    columns = {
        "size": "n_category"
    }
)

In [19]:
two_letter_codes_df

,category,n_category
0,NM,817181
1,NR,135370
2,XM,686862
3,XR,55254


Let's get a feel for exon counts.

In [20]:
# Keep DataFrame.
# Source: https://stackoverflow.com/a/42324086
exon_counts = has_transcript.groupby(['exon_gene', 'exon_transcript_id'], as_index = False).size()
exon_counts = exon_counts.rename(
    columns = {
        "size": "n_exons"
    }
)

In [21]:
exon_counts

,exon_gene,exon_transcript_id,n_exons
0,A1BG,NM_130786.4,8
1,A1BG-AS1,NR_015380.2,4
2,A1CF,NM_001198818.2,14
3,A1CF,NM_001198819.2,15
4,A1CF,NM_001198820.2,14
...,...,...,...
139214,ZZZ3,XM_047417329.1,14
139215,ZZZ3,XM_047417333.1,13
139216,ZZZ3,XM_047417337.1,14
139217,ZZZ3,XM_047417341.1,13


We can immediately identify the number of introns as well.

In [22]:
# Add the intron counts.
exon_counts['n_introns'] = exon_counts['n_exons']-1

In [23]:
exon_counts

,exon_gene,exon_transcript_id,n_exons,n_introns
0,A1BG,NM_130786.4,8,7
1,A1BG-AS1,NR_015380.2,4,3
2,A1CF,NM_001198818.2,14,13
3,A1CF,NM_001198819.2,15,14
4,A1CF,NM_001198820.2,14,13
...,...,...,...,...
139214,ZZZ3,XM_047417329.1,14,13
139215,ZZZ3,XM_047417333.1,13,12
139216,ZZZ3,XM_047417337.1,14,13
139217,ZZZ3,XM_047417341.1,13,12


In [24]:
# # To file.
# exon_counts.to_csv(
#     'exon_intron_counts.tsv',
#     index = False,
#     sep = '\t',
# )

Make a nice table to show the distribution (and therefore necessary work to find transcript lengths) of introns.

In [25]:
# Group to see the distribution of 
# intron counts.
intron_counts = exon_counts.groupby(['n_introns'], as_index = False).size()
intron_counts = intron_counts.rename(
    columns = {
        "size": "n_in_group"
    }
)

In [26]:
intron_counts

,n_introns,n_in_group
0,0,5067
1,1,5352
2,2,11871
3,3,10234
4,4,9300
...,...,...
147,312,2
148,316,1
149,335,2
150,358,1


In [20]:
# # To file.
# intron_counts.to_csv(
#     'intron_counts.tsv',
#     index = False,
#     sep = '\t'
# )

Pick a gene to check the quality of the data.

In [27]:
# TET2 check.
exon_counts[exon_counts['exon_gene'] == 'TET2']

,exon_gene,exon_transcript_id,n_exons,n_introns
120937,TET2,NM_001127208.3,11,10
120938,TET2,NM_017628.4,3,2
120939,TET2,XM_005263082.4,10,9
120940,TET2,XM_006714242.4,9,8
120941,TET2,XM_024454102.2,12,11
120942,TET2,XM_024454103.2,11,10
120943,TET2,XM_047415839.1,6,5
120944,TET2,XM_047415840.1,6,5
120945,TET2,XM_047415841.1,5,4
120946,TET2,XM_047415842.1,5,4


Optionally, sort by the number of introns and write out.

In [28]:
# Sort on number of introns.
exon_counts.sort_values(by = ['exon_gene', 'n_introns'])

,exon_gene,exon_transcript_id,n_exons,n_introns
0,A1BG,NM_130786.4,8,7
1,A1BG-AS1,NR_015380.2,4,3
5,A1CF,NM_001370130.1,12,11
6,A1CF,NM_001370131.1,12,11
11,A1CF,XM_011539729.4,12,11
...,...,...,...,...
139205,ZZZ3,NM_001376154.1,15,14
139208,ZZZ3,NM_015534.6,15,14
139209,ZZZ3,NR_164775.1,15,14
139210,ZZZ3,NR_164776.1,15,14


In [29]:
# Write out...

How about truly binary splice/unspliced genes (useful in scVelo)?

In [30]:
# True binaries.
exon_counts[exon_counts.n_introns <= 1]

,exon_gene,exon_transcript_id,n_exons,n_introns
21,A2M-AS1,NR_137424.1,2,1
22,A2M-AS1,NR_137425.1,2,1
29,A2ML1-AS1,NR_046715.1,2,1
33,A4GALT,NR_146459.3,2,1
65,AADACL4,XM_017001153.1,2,1
...,...,...,...,...
139093,ZSWIM1,XM_005260611.5,2,1
139096,ZSWIM3,NM_080752.4,2,1
139146,ZSWIM9,XM_047438788.1,2,1
139167,ZXDA,NM_007156.5,1,0


In [ ]:
# Write out...